## Assignment || k-NN on  Amazon Reviews Dataset
#### By Rishiraj Adhikary For AppliedAIcourse.com || www.rishiraj.xyz@gmail.com

### Note
#### The amazon data used here is the cleaned and text preprocessed data. Hence, no deduplication or text-preprocessing is done here. Make sure your data is clean and without duplicate entries before you execute this notebook

## Objective
### Evaluating the value of k in k nearest neighbours and checking its accuracy

In [1]:
#initilaization for all the required packages
%matplotlib inline

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc



from sklearn.cross_validation import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import cross_val_score
from collections import Counter
from sklearn import cross_validation

/home/rishi/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# using the SQLite Table to read data.
#final.sqlite is a cleaned deduped and preprocessed data
con = sqlite3.connect('final.sqlite') 


cleaned_data = pd.read_sql_query("""
SELECT *
FROM Reviews
""", con) 

positiveNegative = cleaned_data['Score'] #Keeping labels/class in a different variable so that we can use it latercleaned_data.shape
cleaned_data.head(3)

,index,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
0,138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,b'witti littl book make son laugh loud recit c...
1,138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",b'grew read sendak book watch realli rosi movi...
2,138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,b'fun way children learn month year learn poem...


### We will have to reduce the data sets from 364K to 2K data sets of which 1K is positive and 1K is negative
#### This is done so as to speed up computation. 

In [3]:
#we will take first 1k positive reviews and first 1k negative reviews. Combine them to have a total of 2k text reviews
positiveData = cleaned_data[cleaned_data['Score'] == 'positive']
negativeData = cleaned_data[cleaned_data['Score'] == 'negative']
cleanedData_less = positiveData[:1000].append(negativeData[:1000])
len(cleanedData_less)
#the score corresponding to all 2k reviews
positiveNegativeLabel = cleanedData_less['Score']


### We will sort the data on the basis of the timestamp. This is necessary to do time-based splitting before aplying K-NN

In [4]:
cleanedData_less.sort_values(by=['Time'], inplace=True, kind='quicksort', na_position='last')
positiveNegativeLabel = cleanedData_less['Score']


## Text to Vector Conversion Using Bag of Words. 


In [5]:
#positiveNegativeLabel = cleanedData_less['Score']
count_vect = CountVectorizer() #in scikit-learn
final_counts = count_vect.fit_transform(cleanedData_less['Text'].values)
final_counts.get_shape()
print(type(final_counts))

<class 'scipy.sparse.csr.csr_matrix'>


In [6]:
#we need to convert the sparse matrix to dataframe before we can apply tSNE on the text vectors
final_DF = pd.DataFrame(final_counts.toarray())
type(final_DF)

pandas.core.frame.DataFrame

### Dividing data into train and test set for applying K-NN
SInce the data was sorted on basis of time, hence first 70% of data will be training data and rest30% will be test data. The line below accomplishes that

In [7]:
#70% training data and 30% test data
X_train, X_test, y_train, y_test = cross_validation.train_test_split(final_DF, positiveNegativeLabel, test_size=0.3, random_state=0)

### Applying 10-fold cross validation on training data

In [8]:
# creating odd list of K for KNN
myList = list(range(0,50))
neighbors = list(filter(lambda x: x % 2 != 0, myList))

# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=10, scoring='accuracy') #accuracy measurement, not error
    cv_scores.append(scores.mean())

# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print('\nThe optimal number of neighbors is %d.' % optimal_k)




The optimal number of neighbors is 27.


#### The optimal k will be used as k-NN for the test data set to predict the responnse and evaluate accuracy


In [9]:
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k)

# fitting the model
knn_optimal.fit(X_train, y_train)

# predict the response
pred = knn_optimal.predict(X_test)

# evaluate accuracy
acc = accuracy_score(y_test, pred) * 100
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k, acc))


The accuracy of the knn classifier for k = 27 is 70.333333%


## Result:
Usinng BOW, the accuracy attained is 70.3% with k as 27 in k-NN

## Text to Vector Conversion Using TF-IDF. 


In [9]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(cleanedData_less['Text'].values)
type(final_tf_idf)


scipy.sparse.csr.csr_matrix

In [10]:
#convert the csr_matrix to datsframe so that we can apply t-sne
TFIDF_final_DF = pd.DataFrame(final_tf_idf.toarray())
type(TFIDF_final_DF)

pandas.core.frame.DataFrame

### Dividing data into train and test set for applyting K-NN

In [11]:
### Dividing data into train and test set for applyting K-NN
#70% training data and 30% test data
X_train, X_test, y_train, y_test = cross_validation.train_test_split(TFIDF_final_DF, positiveNegativeLabel, test_size=0.3, random_state=0)


### Applying 3-fold cross validation on training data

In [12]:
# creating odd list of K for KNN
myList = list(range(0,50))
neighbors = list(filter(lambda x: x % 2 != 0, myList))

# empty list that will hold cv scores
cv_scores = []

# perform 10-fold cross validation
for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train, y_train, cv=3, scoring='accuracy') #accuracy measurement, not error
    cv_scores.append(scores.mean())

# changing to misclassification error
MSE = [1 - x for x in cv_scores]

# determining best k
optimal_k = neighbors[MSE.index(min(MSE))]
print('\nThe optimal number of neighbors is %d.' % optimal_k)


The optimal number of neighbors is 47.


#### The optimal k will be used as k-NN for the test data set to predict the responnse and evaluate accuracy


In [13]:
knn_optimal = KNeighborsClassifier(n_neighbors=optimal_k)

# fitting the model
knn_optimal.fit(X_train, y_train)

# predict the response
pred = knn_optimal.predict(X_test)

# evaluate accuracy
acc = accuracy_score(y_test, pred) * 100
print('\nThe accuracy of the knn classifier for k = %d is %f%%' % (optimal_k, acc))


The accuracy of the knn classifier for k = 47 is 81.000000%


## Result:
Usinng TF-IDF, the accuracy attained is 81.0% with k as 47 in k-NN